In [5]:
import os
from sam import chdir_to_repopath

chdir_to_repopath()
from sam.dose_reponse_fit import ModelPredictions
import pandas as pd
import matplotlib.pyplot as plt
from sam.data_formats import read_data, load_datapoints, ExperimentData
import seaborn as sns
from sam.stress_addition_model import (
    OLD_STANDARD,
    sam_prediction,
    FitSettings,
    dose_response_fit,
)
from sam.transforms import Transforms

In [6]:
def generate_settings(data: ExperimentData, transform: Transforms):
    return FitSettings(
        survival_max=data.meta.max_survival,
        param_d_norm=True,
        stress_to_survival=OLD_STANDARD.stress_to_survival,
        survival_to_stress=OLD_STANDARD.survival_to_stress,
        transform=transform,
    )

In [8]:
for path, data, stress, ser in load_datapoints():
    exp_name = os.path.split(path)[1].replace(".xlsx", "")

    ten_points = dose_response_fit(
        ser, generate_settings(data, Transforms.williams_and_linear_interpolation_old)
    )
    eleven_points = dose_response_fit(
        ser, generate_settings(data, Transforms.williams_and_linear_interpolation)
    )

    title = f"{exp_name}_{stress}"

    plt.scatter(
        ten_points.inputs.concentration, ten_points.inputs.survival_rate, label="Data"
    )
    plt.plot(
        ten_points.concentration_curve, ten_points.survival_curve, label="10 Points"
    )
    plt.plot(
        eleven_points.concentration_curve,
        eleven_points.survival_curve,
        label="11 Points",
    )
    plt.xscale("log")
    plt.legend()
    plt.savefig(f"control_imgs/difference_in_linear_transform/{title}.png")
    plt.close()

In [11]:
import zipfile
import os

# List of interesting filenames
interesting = [
    "naeem_Esfenvalerate_2019_Food_1% + Prochloraz_32",
    "naeem_Esfenvalerate_2019_Food_1% + Prochloraz_100",
    "ayesha-cloth_Clothianidin_2024_reference_pre-contamination_Temp_24",
    "naeem_Mix13_2024_Food_1%",
    "naeem_Mix13_2024_Temp_25 + Food_1%",
    "liess_copper_2001_Food_1% + UV",
    "naemm_Clothianidin_2024_reference_Temp_22 + Prochloraz_10",
    "ayesha-cloth_Clothianidin_2024_agricultural_pre-contamination_Temp_24",
    "huang_imidachloprid_2023_Temp_15",
]

# Directory paths
input_directory = "control_imgs/difference_in_linear_transform/"
output_zip = "zips/difference_in_linear_transform.zip"

# Create a zip file
with zipfile.ZipFile(output_zip, "w") as z:
    for file_name in interesting:
        # Form the full file path for each file
        file_path = os.path.join(input_directory, f"{file_name}.png")
        # Check if the file exists before adding it
        if os.path.exists(file_path):
            z.write(file_path, os.path.basename(file_path))